**Данный ноутбук проводит анализ текста с использование Google Cloud Nature Library**

**Расценки на использование данного API:**

Использование вами естественного языка рассчитывается в виде **«единиц»**, где каждый документ, отправляемый в API для анализа, представляет собой **как минимум одну единицу**. Документы, содержащие более 1000 символов Unicode (включая символы пробелов и любые символы разметки, такие как HTML или XML-теги), считаются несколькими единицами, **одна единица на 1000 символов**.

Цены на использование естественного языка рассчитываются **ежемесячно** в зависимости от того, какую функцию API вы использовали, и сколько единиц оценивается с использованием этих функций. В таблице ниже указана цена за 1000 единиц на основе **общего количества единиц, проанализированных в течение расчетного месяца**.

1. - Entity Analysis	Identify entities and label by types such as person, organization, location, events, products and media.
- до 5 тыс единиц - бесплатно, от 5 тыс до 1 млн - 1 долл/1 тыс
2. - Sentiment Analysis	Understand the overall sentiment expressed in a block of text.
- до 5 тыс единиц - бесплатно, от 5 тыс до 1 млн - 1 долл/1 тыс
3. - Entity Sentiment Analysis	Understand the sentiment for entities identified in a block of text.
- до 5 тыс единиц - бесплатно, от 5 тыс до 1 млн - 2 долл/1 тыс
4. - Syntax Analysis	Extract tokens and sentences, identify parts of speech (PoS) and create dependency parse trees for each sentence.
- до 5 тыс единиц - бесплатно, от 5 тыс до 1 млн - 0,5 долл/1 тыс
5. - Content Classification	Identify content categories that apply to a block of text.
- до 30 тыс единиц - бесплатно, от 5 тыс до 1 млн - 2 долл/1 тыс

В данной задаче мы будем использовать для анализа текстовых отзывов:
- Sentiment Analysis (анализ настроений) - для оценки полярности и силы настроений;
- Entity Analysis (анализ сущностей) - для выявления и классификации значимых слов;
- Syntax Analysis (синтаксический анализ) - для разбиения на слова и предложения.

- **После отработаки данного кода API key удален из текста кода в целях безопасности моего Google-аккаунта. Вся полученная в результате обработки инорфмациия сохранена в CSV-файл для дальнейшей работы**
- **Датасет разбивал на мелкие подразделы, чтобы при случайном сбое не потерять данные, за получение которых произведена оплата использования API**
- **Выгрузку полученных данных тоже сделал по частям, чтобы минимизировать размер отдельных файлов для загрузки на гитхаб**

# Импорт библиотек и настройки

In [39]:
import os
import numpy as np
import pandas as pd
from google.cloud import language_v1
from tqdm import tqdm

# этот блок для отслеживания операций с Pandas
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [40]:
PATH = "./data" # относительный путь к подпапке.
pd.set_option('display.max_columns', None)

In [41]:
df = pd.read_csv(os.path.join(PATH, 'order_reviews.csv'), sep=';')
df.shape

(99224, 7)

In [42]:
df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01 00:00:00,2018-03-02 10:26:53


# Предобработка датасета. Прогнозирование затрат на обработку

- Прогнозировали - 42706*3=128118 запросов и сумму расходов 94,3 долл.
- Фактически - 128082 запроса и сумма расходов: 80.65 евро, в т.ч. entity analysis - 32.3 euro, sentimental - 32.3 euro, syntax - 16.13

-

In [43]:
df['C']=df.progress_apply(lambda x: ("" if pd.isna(x['review_comment_title']) else str(x['review_comment_title']) + '. ')\
                                 + ("" if pd.isna(x['review_comment_message']) else str(x['review_comment_message'])), axis=1)

  0%|          | 0/99224 [00:00<?, ?it/s]

In [44]:
count_reviews=df.apply(lambda x: 0 if x['C']=="" else 1, axis=1).sum()
print('Количество отзывов (единиц) для отправки в API: ', count_reviews)
print('Прогнозная стоимость обработки в API, долл: ', (count_reviews-5000)/1000*2.5)

Количество отзывов (единиц) для отправки в API:  42706
Прогнозная стоимость обработки в API, долл:  94.26500000000001


# Функции обработки датасета

In [45]:
# фильтрует датасет и направляет для обработки в API ненулевые значения, чтобы не платить за пустые запросы.
# и она же вызывает три функции API Google
def get_text_for_nlp(message):
    if len(message)<2:
        #print(0)
        return 0
    else:
        #print(message)
        sent_score, sent_magnitude=analyze_text_sentiment(message) # два значения 
        entities_list=analyze_text_entities(message) # список со словарями сущностей
        sent_count, token_count, sentlist, tokenlist =analyze_text_syntax2(message) # синтаксический анализ
        return sent_score, sent_magnitude, entities_list, sent_count, token_count, sentlist, tokenlist

In [46]:
def analyze_text_sentiment(text):
    try:
        client = language_v1.LanguageServiceClient.from_service_account_json('celestial-digit-0000000000.json')
        document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
        response = client.analyze_sentiment(document=document)
        sentiment = response.document_sentiment
        return sentiment.score, sentiment.magnitude
    except Exception:
        print('Ошибка в функции sentiment')  
        return 'НЕТ', 'НЕТ'

In [47]:
def analyze_text_entities(text):
    try:
        client = language_v1.LanguageServiceClient.from_service_account_json('celestial-digit-00000000000.json')
        document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
        response = client.analyze_entities(document=document)
        sss=[]
        for entity in response.entities:
            results = dict(
                name=entity.name,
                type=entity.type_.name,
                salience=entity.salience,
                wikipedia_url=entity.metadata.get("wikipedia_url", "-"),
                mid=entity.metadata.get("mid", "-"),
            )
            sss.append(results)

        return sss
    except Exception:
        print('Ошибка в функции entities')  
        return 'НЕТ'

In [48]:
def analyze_text_syntax2(text):
    try:
        client = language_v1.LanguageServiceClient.from_service_account_json('celestial-digit-00000000000.json')
        document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

        response = client.analyze_syntax(document=document)

        sent_count=len(response.sentences)
        token_count=len(response.tokens)
        sentlist=[]
        for sentence in response.sentences:
            sentlist.append(sentence.text.content)
        tokenlist=[]    
        for token in response.tokens:
            results = dict(
                token_text=token.text.content,
                token_label=token.dependency_edge.label.name,
                token_head_index=token.dependency_edge.head_token_index,
                token_tag=token.part_of_speech.tag.name,
                token_gender=token.part_of_speech.gender.name,
                token_number=token.part_of_speech.number.name,
                token_proper=token.part_of_speech.proper.name,
                token_lemma=token.lemma,
            )
            tokenlist.append(results)    
        return sent_count, token_count, sentlist, tokenlist
    except Exception:
        print('Ошибка в функции syntax2')  
        return 'НЕТ', 'НЕТ','НЕТ','НЕТ'

# Запуск обработки

Так как это долгий процесс на моем оборудовании, а также в связи с возникновением сбоев на стороне API и чтобы не потерять обработанную информацию, то датасет разбивал на очень маленькие разделы для отправления в обработку.

**Итого обработка заняла почти сутки**

In [55]:
#for i in tqdm(range(1,43)):
df['tmp']=np.nan
for i in tqdm(range(1,43)):    
    print('c ',(i-1)*1000, ' до включительно ',i*1000)
    print(df['review_id'][(i-1)*1000],'---',df['review_id'][i*1000],'---',df['review_id'][(i-1)*1000:i*1000].shape)
    df['tmp'][(i-1)*1000:i*1000] = df['C'][(i-1)*1000:i*1000].progress_apply(get_text_for_nlp)
    df5=df[['review_id','order_id','tmp']]
    filename1='tmp'+str(i)+'.csv'
    df5[(i-1)*1000:i*1000].to_csv(os.path.join(PATH, filename1), sep=';', header=True, index=False)

  0%|          | 0/42 [00:00<?, ?it/s]

c  0  до включительно  1000
7bc2406110b926393aa56f80a40eba40 --- 05c968cc75440c647f58e6bfe1ee15bb --- (1000,)


  0%|          | 0/1000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
  2%|▏         | 1/42 [12:46<8:43:57, 766.77s/it]

c  1000  до включительно  2000
05c968cc75440c647f58e6bfe1ee15bb --- 7629603f1f362dd403eb621d7653c896 --- (1000,)


...

  0%|          | 0/1000 [00:00<?, ?it/s]

 98%|█████████▊| 41/42 [8:21:09<09:52, 592.09s/it]

c  41000  до включительно  42000
8ef30b486d408a19f5fea3e6dac8cc97 --- 6452911fae3dad5d7dd8553ac35248e2 --- (1000,)


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 42/42 [8:31:14<00:00, 730.34s/it]


In [56]:
for i in tqdm(range(43,100)):    
    print('c ',(i-1)*1000, ' до включительно ',i*1000)
    print(df['review_id'][(i-1)*1000],'---',df['review_id'][i*1000],'---',df['review_id'][(i-1)*1000:i*1000].shape)
    df['tmp'][(i-1)*1000:i*1000] = df['C'][(i-1)*1000:i*1000].progress_apply(get_text_for_nlp)
    df5=df[['review_id','order_id','tmp']]
    filename1='tmp'+str(i)+'.csv'
    df5[(i-1)*1000:i*1000].to_csv(os.path.join(PATH, filename1), sep=';', header=True, index=False)

  0%|          | 0/57 [00:00<?, ?it/s]

c  42000  до включительно  43000
6452911fae3dad5d7dd8553ac35248e2 --- 126dba0769720a05295360be53b92669 --- (1000,)


  0%|          | 0/1000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
  2%|▏         | 1/57 [15:59<14:55:52, 959.86s/it]

c  43000  до включительно  44000
126dba0769720a05295360be53b92669 --- a40a0e99764674bc313061cec03c801c --- (1000,)


...

 98%|█████████▊| 56/57 [10:51:36<11:33, 693.10s/it]

c  98000  до включительно  99000
bebf8e035a900f0759ead21eca6b96a2 --- 11fcd0e723918cabcf2c2cbe4a8600d7 --- (1000,)


  0%|          | 0/1000 [00:00<?, ?it/s]

Ошибка в функции entities
Ошибка в функции syntax2


100%|██████████| 57/57 [11:02:40<00:00, 697.56s/it]


In [59]:
#for i in tqdm(range(99000,99225)):    
#    print('c ',(i-1), ' до включительно ',i)
#    print(df['review_id'][(i-1)],'---',df['review_id'][i],'---',df['review_id'][(i-1):i].shape)
df['tmp'][99000:99225] = df['C'][99000:99225].progress_apply(get_text_for_nlp)
df5=df[['review_id','order_id','tmp']]
filename1='tmp'+str(100)+'.csv'
df5[99000:99225].to_csv(os.path.join(PATH, filename1), sep=';', header=True, index=False)

  0%|          | 0/224 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Сохраняем промежуточные результаты, разворачиваем полученные данные по столбцам, сохраняем в CSV-файлы в виде разных наборов полей

In [61]:
# сохраняем промежуточный результат от API во внешний файл, с привязкой к ключевым полям.
filename1='tmp_all.csv'
df5.to_csv(os.path.join(PATH, filename1), sep=';', header=True, index=False)

In [75]:
df[['C','tmp']][0:10]

C  \
0                                                      
1                                                      
2                                                      
3              Recebi bem antes do prazo estipulado.   
4  Parabéns lojas lannister adorei comprar pela I...   
5                                                      
6                                                      
7                                                      
8                                                      
9  recomendo. aparelho eficiente. no site a marca...   

                                                 tmp  
0                                                  0  
1                                                  0  
2                                                  0  
3  (0.8999999761581421, 0.8999999761581421, [{'na...  
4  (0.8999999761581421, 0.8999999761581421, [{'na...  
5                                                  0  
6                                                  0  
7                                                  0  
8                                                  0  
9  (0.10000000149011612, 0.10000000149011612, [{'...

In [91]:
# необработанные тексты отзывов. Это однобуквенные значения.
df[(df['C']!="") & (df['tmp']==0)][['C','tmp']].shape

(76, 2)

In [101]:
df5[['review_id','order_id','tmp']][15:21]

,review_id,order_id,tmp
15,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,"(0.8999999761581421, 1.7999999523162842, [{'name': 'Super', 'type': 'PERSON', 'salience': 0.326386034488678, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'entrega', 'type': 'EVENT', 'salience': 0.18561504781246185, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'Vendedor', 'type': 'PERSON', 'salience': 0.18386904895305634, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'prazo', 'type': 'OTHER', 'salience': 0.17040300369262695, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'produto', 'type': 'CONSUMER_GOOD', 'salience': 0.13372687995433807, 'wikipedia_url': '-', 'mid': '-'}], 2, 14, [Super recomendo., Vendedor confiável, produto ok e entrega antes do prazo.], [{'token_text': 'Super', 'token_label': 'NSUBJ', 'token_head_index': 1, 'token_tag': 'NOUN', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'PROPER', 'token_lemma': 'Super'}, {'token_text': 'recomendo', 'token_label': 'ROOT', 'token_head_index': 1, 'token_tag': 'VERB', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'recomendar'}, {'token_text': '.', 'token_label': 'P', 'token_head_index': 1, 'token_tag': 'PUNCT', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': '.'}, {'token_text': 'Vendedor', 'token_label': 'ROOT', 'token_head_index': 3, 'token_tag': 'NOUN', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'PROPER', 'token_lemma': 'Vendedor'}, {'token_text': 'confiável', 'token_label': 'AMOD', 'token_head_index': 3, 'token_tag': 'ADJ', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'confiável'}, {'token_text': ',', 'token_label': 'P', 'token_head_index': 3, 'token_tag': 'PUNCT', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': ','}, {'token_text': 'produto', 'token_label': 'CONJ', 'token_head_index': 3, 'token_tag': 'NOUN', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'produto'}, {'token_text': 'ok', 'token_label': 'AMOD', 'token_head_index': 6, 'token_tag': 'ADJ', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'ok'}, {'token_text': 'e', 'token_label': 'CC', 'token_head_index': 3, 'token_tag': 'CONJ', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': 'e'}, {'token_text': 'entrega', 'token_label': 'CONJ', 'token_head_index': 3, 'token_tag': 'NOUN', 'token_gender': 'FEMININE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'entrega'}, {'token_text': 'antes', 'token_label': 'ADVMOD', 'token_head_index': 3, 'token_tag': 'ADV', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': 'antes'}, {'token_text': 'do', 'token_label': 'PREP', 'token_head_index': 10, 'token_tag': 'ADP', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'do'}, {'token_text': 'prazo', 'token_label': 'POBJ', 'token_head_index': 11, 'token_tag': 'NOUN', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'prazo'}, {'token_text': '.', 'token_label': 'P', 'token_head_index': 3, 'token_tag': 'PUNCT', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': '.'}])"
16,9314d6f9799f5bfba510cc7bcd468c01,0dacf04c5ad59fd5a0cc1faa07c34e39,"(-0.10000000149011612, 0.10000000149011612, [{'name': 'ESSA', 'type': 'CONSUMER_GOOD', 'salience': 0.2512953281402588, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'AGORA', 'type': 'OTHER', 'salience': 0.23210185766220093, 'wikipedia_url': '-', 'mid': '-'}, {'name': 'DECPCIONOU', 'type': 'CONSUMER_GOOD

In [109]:
df3=df.tmp.progress_apply(pd.Series) # variant 2

  0%|          | 0/99224 [00:00<?, ?it/s]

In [110]:
df3.rename(columns={0:'sent_score',1:'sent_magnitude', 2:'entities_list',3:'sentences_count',4:'token_count',5:'sentlist',6:'tokenlist'},  inplace=True)

In [111]:
df3.shape

(99224, 7)

In [117]:
df3.to_csv(os.path.join(PATH, 'tmp_all_just.csv'), sep=';', header=True, index=False)

In [116]:
df3.head(4)

,sent_score,sent_magnitude,entities_list,sentences_count,token_count,sentlist,tokenlist
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0.9,0.9,"[{'name': 'prazo', 'type': 'OTHER', 'salience': 1.0, 'wikipedia_url': '-', 'mid': '-'}]",1,7,[Recebi bem antes do prazo estipulado.],"[{'token_text': 'Recebi', 'token_label': 'ROOT', 'token_head_index': 0, 'token_tag': 'VERB', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'Receber'}, {'token_text': 'bem', 'token_label': 'ADVMOD', 'token_head_index': 2, 'token_tag': 'ADV', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': 'bem'}, {'token_text': 'antes', 'token_label': 'ADVMOD', 'token_head_index': 0, 'token_tag': 'ADV', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': 'antes'}, {'token_text': 'do', 'token_label': 'PREP', 'token_head_index': 2, 'token_tag': 'ADP', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'do'}, {'token_text': 'prazo', 'token_label': 'POBJ', 'token_head_index': 3, 'token_tag': 'NOUN', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'prazo'}, {'token_text': 'estipulado', 'token_label': 'AMOD', 'token_head_index': 4, 'token_tag': 'ADJ', 'token_gender': 'MASCULINE', 'token_number': 'SINGULAR', 'token_proper': 'NOT_PROPER', 'token_lemma': 'estipulado'}, {'token_text': '.', 'token_label': 'P', 'token_head_index': 0, 'token_tag': 'PUNCT', 'token_gender': 'GENDER_UNKNOWN', 'token_number': 'NUMBER_UNKNOWN', 'token_proper': 'NOT_PROPER', 'token_lemma': '.'}]"


In [115]:
df4=pd.concat([df,df3],ignore_index=False, axis=1)
print(df4.shape)

(99224, 16)


## Это финальный файл для дальнейшей работы - ключевые поля 'review_id','order_id' и развернутые данные из API

На всякий случай сохраняю с двумя разными разделителями. А также полный файл и сокращенные. На гитхаб эти файлы в оригинальном виде не поместяться

In [123]:
columns_for_used=['review_id','order_id','sent_score','sent_magnitude', 'entities_list','sentences_count','token_count','sentlist','tokenlist']
columns_for_used_num=['review_id','order_id','sent_score','sent_magnitude', 'sentences_count','token_count']
columns_for_used_entit=['review_id','order_id','entities_list']
columns_for_used_sent=['review_id','order_id','sentlist']
columns_for_used_token=['review_id','order_id','tokenlist']

In [120]:
df4.to_csv(os.path.join(PATH, 'order_reviews_only_Google_full.csv'), sep=';', header=True, index=False)

In [121]:
df4.to_csv(os.path.join(PATH, 'order_reviews_only_Google_full2.csv'), sep='|', header=True, index=False)

In [119]:
df4[columns_for_used].to_csv(os.path.join(PATH, 'order_reviews_only_Google1.csv'), sep=';', header=True, index=False)

In [122]:
df4[columns_for_used].to_csv(os.path.join(PATH, 'order_reviews_only_Google2.csv'), sep='|', header=True, index=False)

In [124]:
df4[columns_for_used_num].to_csv(os.path.join(PATH, 'order_reviews_only_Google_num1.csv'), sep=';', header=True, index=False)
df4[columns_for_used_num].to_csv(os.path.join(PATH, 'order_reviews_only_Google_num2.csv'), sep='|', header=True, index=False)

In [125]:
df4[columns_for_used_entit].to_csv(os.path.join(PATH, 'order_reviews_only_Google_entit1.csv'), sep=';', header=True, index=False)
df4[columns_for_used_entit].to_csv(os.path.join(PATH, 'order_reviews_only_Google_entit2.csv'), sep='|', header=True, index=False)

In [126]:
df4[columns_for_used_sent].to_csv(os.path.join(PATH, 'order_reviews_only_Google_sent1.csv'), sep=';', header=True, index=False)
df4[columns_for_used_sent].to_csv(os.path.join(PATH, 'order_reviews_only_Google_sent2.csv'), sep='|', header=True, index=False)

In [127]:
df4[columns_for_used_token].to_csv(os.path.join(PATH, 'order_reviews_only_Google_token1.csv'), sep=';', header=True, index=False)
df4[columns_for_used_token].to_csv(os.path.join(PATH, 'order_reviews_only_Google_token2.csv'), sep='|', header=True, index=False)